# Bing Search 기반의 RAG를 활용한 데모

이 데모에서는 GPT3 모델을 사용하여 자연 쿼리를 분석하고 지식 기반을 사용하여 더 많은 정보를 검색하고 질문에 답변하는 방법을 보여줍니다.  

해당 실습을 위해서는 [Bing 리소스](https://portal.azure.com/#blade/HubsExtension/BrowseResourceBlade/resourceType/Microsoft.Bing%2Faccounts)를 만들고 API Key를 [.env](../.env) 파일에 입력해야 합니다. 

1. Bing Search API 활용: 검색한 결과를 참고하는 방법을 소개합니다. 여기서는 Bing Search API를 활용합니다.

In [1]:
import os
import json
import requests
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv('DEPLOYMENT_NAME')

## 1. Bing Search API를 활용하는 방법

### Start with a natural question  
다음과 같이 LLM이 가지고 있지 않을 수 있는 데이터에 대한 접근이 필요할 때, 검색엔진을 활용할 수 있습니다.

### Step 1: GPT3: What do I need to to answer this question?

In [2]:
# 검색을 위해 필요한 정보를 추출하는 함수 정의
def get_search_info(input_query):
    system_prompt='''
    Answer the following questions as best you can. You have access to the following tools: You can only Korean language.
    Web Search: Use the web to find information
    Question: Natural language-based questions that users want to ask through web searches
    Answer: Describes how to effectively answer people's natural language-based questions and creates optimized search keywords needed for use in the Bing Search API. And create optimized search keywords needed for use in the Bing Search API. The answer format must strictly follow the JSON format below.
    {"Keywords": "Specify three sets of keywords information to search for key data that matches the user's question intent."}
    '''
    user_prompt = f'''
    Question: {input_query}
    '''

    # Azure OpenAI GPT를 프롬프트 엔지니어링의 Few-shot 예시
    prompt = [{"role":"system", "content":system_prompt},
                {"role":"user", "content":"Question: 2024년 한국 총선 날짜가 언제인지 회사를 안 가도 되는지 알려줘."},
                {"role":"assistant", "content":"""{"Keywords": "2024년 한국 총선 날짜 + 2024년 총선 공휴일 여부 + 총선 투표일 휴무 정책"}"""},
                {"role":"user","content": user_prompt}
            ]
        
    response = client.chat.completions.create(
        model=deployment_name,
        messages = prompt,
        max_tokens=600,
        response_format={"type": "json_object"}
    )

    keywords = json.loads(response.choices[0].message.content)['Keywords']
    
    # 만약 keywords가 list 형태일 경우, '+' 문자로 join하여 반환
    if type(keywords) == list:
        keywords = ' + '.join(keywords)

    return json.loads(response.choices[0].message.content)['Keywords']

### Step 2: Search web for more details  

Bing Search API와 관련된 정보는 다음 Page에서 확인할 수 있습니다.  
https://www.microsoft.com/en-us/bing/apis/bing-web-search-api

In [3]:
# Bing search API를 이용하여 검색 결과를 가져오는 함수 정의
def get_search_result(keywords):
    subscription_key = os.getenv("BING_SEARCH_KEY")
    endpoint = os.getenv("BING_SEARCH_ENDPOINT") + "/v7.0/search"

    query = keywords

    # Construct a request
    mkt = 'ko_KR'
    params = { 'q': query, 'mkt': mkt }
    headers = { 'Ocp-Apim-Subscription-Key': subscription_key }

    # Call the API
    response = requests.get(endpoint, headers=headers, params=params)

    context = ''
    for result in response.json()['webPages']['value']:
        context += result['snippet'] + '\n'

    return context

In [4]:
# 검색 결과를 기반으로 사용자의 질문에 대한 답을 생성하는 함수 정의
def generate_answer(input_query, context):
    system_prompt = """You are an agent who answers the user's questions based on the data provided. You can only Korean language."""
    user_prompt = f"""Question: {input_query}
    Web Search Results: {context}
    Answer: 
    """

    response = client.chat.completions.create(
        model=deployment_name,
        messages = [
            {"role":"system", "content":system_prompt},
            {"role":"user","content": user_prompt}
        ],
        max_tokens=600
    )

    return response.choices[0].message.content

In [5]:
# 사용자로부터 검색 질의어를 입력받아서 결과를 출력하는 함수 정의
def get_input_query(input_query):
    # 검색 질의어 출력
    print("-"*80)
    print("검색질의어: " + input_query)
    print("-"*80)

    # 검색 키워드를 추출하는 함수 호출
    keywords = get_search_info(input_query)
    print("검색키워드: " + keywords)
    print("-"*80)

    # 검색 결과를 가져오는 함수 호출
    context = get_search_result(keywords)
    print("검색결과: " + context)
    print("-"*80)

    # 사용자에게 제공할 답변을 생성하는 함수 호출
    answer = generate_answer(input_query, context)
    print("최종답변: " + answer)
    print("-"*80)

In [7]:
input_query = "2025년 '부처님 오신 날'이 언제야? 쉬는 날 맞아? 공휴일이면 대체 휴무는 언제야?"
get_input_query(input_query)

--------------------------------------------------------------------------------
검색질의어: 2025년 '부처님 오신 날'이 언제야? 쉬는 날 맞아? 공휴일이면 대체 휴무는 언제야?
--------------------------------------------------------------------------------
검색키워드: 2025년 부처님 오신 날 날짜 + 부처님 오신 날 공휴일 여부 + 대체 휴무 날짜 2025
--------------------------------------------------------------------------------
검색결과: 대체공휴일은 설·추석 연휴의 경우 일요일과, 어린이날과 국경일인 3·1절, 광복절, 개천절, 한글날, 부처님 오신 날, 성탄절은 토·일요일과 겹치는 경우에 적용됩니다.
2025년 공휴일 한눈에 ... 👉 부처님 오신 날. ... 위와 같이 2025년의 공휴일 및 대체공휴일이 지정되어 있으며, 삼일절과 부처님 오신날에 대한 대체공휴일이 2일 발생하여 평일에는 총 15일의 공휴일이 발생하게 됩니다. ...
부처님 오신 날은 한국에서 널리 기념하는 공휴일입니다. 이전 연도의 날짜를 보려면 페이지 하단으로 스크롤하십시오. 휴일은 중국 음력에 따라 네 번째 달의 여덟 번째 일 (日)을 축하됩니다. 이 휴가는 4월 말에서 5월초 사이에 열립니다. 한국에서 부처님 오신 날의 기념은 한국 문화에 영향을 주는 불교의 영향을 강력히 대표합니다. 한국 인구 중 약 20%만이 불교이지만, 부처님 오신 날은 공휴일로 인정되었습니다. 부처님 오신 날은 탄생을 축하하는 것이지만, 불교를 만든 싯다르타의 계몽을 축하는 날이기도 합니다. 한국과 함께, 부처님 오신날은 동남 아시아의 여러 나라에서도 축하합니다.
설날, 삼일절, 어린이날 및 부처님 오신 날, 추석 등 주요 공휴일에 대체공휴일이 적용되어 최소 3일 연속 휴일이 보장됩니다. 아래표를 통해 

In [8]:
input_query = "2025년 구정 설 연휴는 언제야? 연휴 기간은으로 얼만큼 쉴 수 있을지 알려줘."
get_input_query(input_query)

--------------------------------------------------------------------------------
검색질의어: 2025년 구정 설 연휴는 언제야? 연휴 기간은으로 얼만큼 쉴 수 있을지 알려줘.
--------------------------------------------------------------------------------
검색키워드: 2025년 구정 설 연휴 날짜 + 2025년 설 연휴 기간 + 2025년 구정 휴일 정보
--------------------------------------------------------------------------------
검색결과: 2025년 설 연휴: 1월 28일, 29일, 30일. 2025 추석 연휴 언제? 2025년 구정 연휴는 아쉽게되었지만 추석은 얘기가 다르다. 2025년 달력에서 10월을 확인해보자. 빨간색 날짜만봐도 무척 긴 연휴가 될거라는 것을 알 수 있다. 추석 당일은 10월 6일 월요일이다. 그런데 추석 연휴인 10월 5일이 일요일과 겹치기때문에 대체공휴일이 적용되어 10월 8일이 추가로 빨간날이 되었다. 여기에 앞 뒤로 10월 3일 개천절에 10월 9일 한글날까지 이어져 총 7일 연휴가 가능해졌다. 만약 10월 10일을 휴가 낼 수 있다면 무려 총 10일의 휴일이 될 수 있다.
4. 2025년 공휴일 통계 • 총 공휴일: 18일 • 주말 포함 총 휴일: 119일 • 대체공휴일: 3일 • 가장 긴 연휴: 10월 추석 + 한글날 연휴 (최대 10일) 5. 연휴를 활용한 추천 활동 1. 국내 여행지 추천: • 설날 연휴: 제주도, 부산, 강릉 (겨울 명소)
안녕하세요! 😊 이번 포스팅에서는 2025년 법정 공휴일과 대체공휴일, 그리고 명절 연휴 정보를 자세히 정리했습니다. 연휴를 잘 활용해 알찬 계획을 세워보세요! 또한, 아래 유용한 링크를 클릭하면 더 자세한 정보를 확인하실 수 있습니다. 📌 2025년 설날 및 추석 연휴 상세 안내2025년에